In [1]:
#| default_exp fmf_biobert

# Found My Fitness Example
>"Here we'll try using elements from the biobertApi module to label biomedical terms in the comments section of a Found My Fitness Youtube video"

In [2]:
#|hide
# import biobert-bern 
from biobert_bern.biobertApi import *

In [3]:
!pip install pathlib

In [4]:
import requests
import re
import pandas as pd
from nbdev.showdoc import *
import fastai
from pathlib import *
import numpy as np     

In [5]:
path = Path('../..')
comment_csv= 'found_my_fitness_UCWF8SqJVNlx-ctXbLswcTcA_youtube_comments_only_122rows.csv'

In [6]:
coms = pd.read_csv(path/comment_csv)

In [7]:
coms.rename(columns = {'comment': 'comments'},inplace=True)

In [8]:
coms.tail()

,vid_id,vid_channel_id,vid_deets,vid_response,title,publishedAt,tcomment_id,tcresponse,textDisplay,textOriginal,totalReplyCount,rep_comment,reply_id,comments
117,g4QqJoox8tc,NaN,NaN,NaN,NaN,2022-11-02T03:21:53Z,Ugy2ZO2t370rDO-eWoZ4AaABAg,"{'kind': 'youtube#commentThread', 'etag': '4vV...",How to combat increased food intake after quit...,How to combat increased food intake after quit...,3.0,NaN,NaN,How to combat increased food intake after quit...
118,g4QqJoox8tc,NaN,NaN,NaN,NaN,2022-11-02T15:43:40Z,NaN,"{'kind': 'youtube#comment', 'etag': 'GB9aSyxEx...",It's brutal. Get the garbage food out of your ...,It's brutal. Get the garbage food out of your ...,NaN,"{'kind': 'youtube#comment', 'etag': 'GB9aSyxEx...",Ugy2ZO2t370rDO-eWoZ4AaABAg.9husbe2WJcN9hwCVh8KBNj,It's brutal. Get the garbage food out of your ...
119,g4QqJoox8tc,NaN,NaN,NaN,NaN,2022-11-02T14:04:02Z,NaN,"{'kind': 'youtube#comment', 'etag': 'bs6xBMC7h...",go animal based,go animal based,NaN,"{'kind': 'youtube#comment', 'etag': 'bs6xBMC7h...",Ugy2ZO2t370rDO-eWoZ4AaABAg.9husbe2WJcN9hw15tjmPLT,go animal based
120,g4QqJoox8tc,NaN,NaN,NaN,NaN,2022-11-02T11:20:13Z,NaN,"{'kind': 'youtube#comment', 'etag': 'yd5KSzb3_...",... and they said it gives you the munchies.,... and they said it gives you the munchies.,NaN,"{'kind': 'youtube#comment', 'etag': 'yd5KSzb3_...",Ugy2ZO2t370rDO-eWoZ4AaABAg.9husbe2WJcN9hvjM9rBbfr,... and they said it gives you the munchies.
121,g4QqJoox8tc,NaN,NaN,NaN,NaN,2022-11-02T03:07:55Z,Ugx3lPLxH7NyOIZxzEl4AaABAg,"{'kind': 'youtube#commentThread', 'etag': 'Gzv...",My first thought was Rich Roll,My first thought was Rich Roll,0.0,NaN,NaN,My first thought was Rich Roll


# Create Text Block
> Here we'll put all the text together

In [9]:
# text_block = create_text_block(coms);# text_block

In [10]:
# text_block[:5000]

### Get Comment Spans

In [11]:
# dfi = get_comment_spans_textblock(text_block); dfi.tail(3)

In [12]:
len(text_block)

NameError: name 'text_block' is not defined

In [ ]:
range(len(text_block))

In [ ]:
for i in range(5000, len(text_block), 5000):
    quotient = i / 5000
    print(f"{i} divided by 5000 is {int(quotient)}.")
    

### Create Section Labels

In [14]:
coms['textLen'] = coms.comments.apply(lambda row: len(row))
coms['cumTxtLen'] = coms.textLen.cumsum()
coms['section'] = np.ceil(coms.cumTxtLen/5000) # round all values up 
coms.section = coms.section.astype(int)

In [17]:
blocks = pd.DataFrame()
for o in range(1,int(coms.section.max()+1)):
    print(o)
    df = coms.loc[coms.section == o]
#     print(df.head(1))
    df = df[['comments']]
    text_block = create_text_block(df);# text_block    
    
    blocks = blocks.append({'text_block':text_block,'section':o},ignore_index=True)

1
2
3
4
5


In [19]:
df

,idx,comments,comidx
0,94,"Hello, \u200bbone broth help a lot with alcool...",0 94\n1 95\n2 96\n3 97\n4 ...
1,95,The next time you have low level anxiety and a...,0 94\n1 95\n2 96\n3 97\n4 ...
2,96,People in the military will prove you wrong😂,0 94\n1 95\n2 96\n3 97\n4 ...
3,97,People in the military exercise about as vigor...,0 94\n1 95\n2 96\n3 97\n4 ...
4,98,Thank you for this presentation.,0 94\n1 95\n2 96\n3 97\n4 ...
5,99,"I have to force myself, both to do exercise an...",0 94\n1 95\n2 96\n3 97\n4 ...
6,100,Rhonda Patrick instant like crew checking in. ...,0 94\n1 95\n2 96\n3 97\n4 ...
7,101,"Thanks, Grant!",0 94\n1 95\n2 96\n3 97\n4 ...
8,102,Watching while working thru a hangover……… good...,0 94\n1 95\n2 96\n3 97\n4 ...
9,103,I haven't had a (hardly) sip since last Christ...,0 94\n1 95\n2 96\n3 97\n4 ...


In [18]:
blocks.text_block[0]

'0      0\n1      1\n2      2\n3      3\n4      4\n5      5\n6      6\n7      7\n8      8\n9      9\n10    10\n11    11\n12    12\n13    13\n14    14\n15    15\n16    16\n17    17\n18    18\n19    19\n20    20\n21    21\n22    22\n23    23\n24    24\n25    25\n26    26\n27    27\nname: idx, dtype: object:: if exercise and/or the interaction with alcohol behaviors has really piqued your interest, make sure to check out my recent "sober october" post series found on my instagram: https://www.instagram.com/foundmyfitness support the show as a premium member: https://www.foundmyfitness.com/crowdsponsor\\n\\nthanks for watching! :: 0      0\n1      1\n2      2\n3      3\n4      4\n5      5\n6      6\n7      7\n8      8\n9      9\n10    10\n11    11\n12    12\n13    13\n14    14\n15    15\n16    16\n17    17\n18    18\n19    19\n20    20\n21    21\n22    22\n23    23\n24    24\n25    25\n26    26\n27    27\nname: idx, dtype: object:: hi rhonda, could this mean that "one time" phsychedelic tr

In [ ]:
blocks.text_block[0]

In [ ]:
blocks[0][0][0]

In [ ]:
totalLen = 0
for row in dfit.iterrows():
    totalLen = totalLen + len(b[0])
    print(b[0])
    print()
    print(len(b[0]))
    print(totalLen)
    
    
    

In [ ]:
i = 0
for index, row in df.iterrows():
    if i>len(df.col):
        break
    else:
        f = open( str(i) + '.txt', 'w', encoding='utf-8')
        f.write(row[0])
        f.close()
        i+=1


In [ ]:
# Send it all to Bern2
output = query_plain(text=text_block);  

In [ ]:
output

## Try by comment

In [ ]:
coms.comments

In [ ]:
df = pd.DataFrame()
for o in coms.comments[:1]:
    out = query_plain(text=o)
    df = df.append({'out':out},ignore_index=True)
df.tail()

In [ ]:
df2 = pd.DataFrame()
for p,o in zip(df.index, df.out):
    dft = pd.DataFrame(o['annotations'])
    dft['comidx'] = p
    df2 = df2.append(dft)
df2
    

In [ ]:
pd.DataFrame(df['annotations'])